In [1]:
! pip install semantic-kernel

In [4]:
import semantic_kernel as sk
from dotenv import dotenv_values

config = dotenv_values()
kernel = sk.Kernel()
service_id = "preserve_chat_history"

In [ ]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import AzureChatPromptExecutionSettings

azure_chat_service = AzureChatCompletion(
    service_id=service_id,
    deployment_name=config["AZURE_OPENAI_DEPLOYMENT"],
    api_version=config["AZURE_API_VERSION"],
    api_key=config["AZURE_OPENAI_API_KEY"],
    endpoint=config["AZURE_OPENAI_ENDPOINT"]
)

kernel.add_service(azure_chat_service)

azure_execution_settings = AzureChatPromptExecutionSettings(
    service_id=service_id,
    ai_model_id=config["AZURE_OPENAI_DEPLOYMENT"],
    max_tokens=1000,
    temperature=0.7
    # other settings can come here too
)

In [9]:
#prompt template - can add rules, do's and don'ts
prompt = """
You're a chatbot who know a lot of apricot riddles.

{{$history}}
User: {{$user_input}}
AI:
"""

In [ ]:
import semantic_kernel as sk
from semantic_kernel.functions import KernelArguments
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

prompt_template_config =  PromptTemplateConfig(
    template=prompt,
    name="apricot_riddle_teller",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
        InputVariable(name="history", description="The chat history", is_required=True)
    ],
    execution_settings=azure_execution_settings
)

In [11]:
from semantic_kernel.contents.chat_history import ChatHistory

chat_function = kernel.create_function_from_prompt(
    function_name="chat_helper",
    plugin_name="apricotRiddlePlugin",
    prompt_template_config=prompt_template_config
)

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good in telling riddles and jokes.")

In [12]:
async def chat_helper(input_text: str) -> None:
    chat_history.add_user_message(input_text)

    arguments = KernelArguments(
        user_input=input_text,
        history=chat_history
    )
    answer = await kernel.invoke(chat_function, arguments=arguments)
    print(answer)

    chat_history.add_assistant_message(str(answer))

In [13]:
user_input = "Can you tell me a one liner apricot riddle without telling its answer."
arguments = KernelArguments(user_input=user_input, history=chat_history)
response = await kernel.invoke(chat_function, arguments=arguments)
chat_history.add_assistant_message(str(response))
print(response)

What fruit can be found in the sun and also has a pit but isn’t a rock?


In [14]:
await chat_helper("Can you tell the anser of the riddle you just mentioned?")

The answer to the riddle is an apricot! It’s a fruit that grows in sunny places and has a pit inside. Would you like to hear another riddle?


In [17]:
await chat_helper("Tell me another ONE LINE RIDDLE, topic being an animal, do not tell me the answer!")

I’m known for my song and my colorful feathers,  
In orchards I chirp, where the apricot gathers.  
What am I?


In [19]:
await chat_helper("are you a parrot! Tell me the answer!")

Great guess, but the answer is actually a songbird! They love to sing in orchards, including where apricots grow. Would you like to try another riddle?
